In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file and return it as a DataFrame."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        return pd.DataFrame(data)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame(columns=["data", "labels"])


def save_custom_dictionary(filename, new_vocabulary):
    """Save or update the custom dictionary."""
    if os.path.exists(filename):
        with open(filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        new_vocabulary.update(existing_vocabulary)
    
    with open(filename, "w") as file:
        json.dump(list(new_vocabulary), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions for a list of words."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [list(spell.candidates(word)) for word in words_to_check]
    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer for character n-grams."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    spell = SpellChecker(language="fr")
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    misclassified_samples = []
    misclassification_threshold = 5
    epochs = 0

    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df.empty:
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)
        model = train_model(X_train_vectorized, all_y_train)

        words_to_check = ["kng", "hgost", "mney", "hllo", "temle", "lon", "aler"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        misclassified_samples.extend(
            [word for word, prediction in zip(words_to_check, predictions) if prediction == 0]
        )

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    save_custom_dictionary(custom_dictionary_filename, new_vocabulary)

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(words_to_check, corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()


KeyError: 'labels'

In [3]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file and return it as a DataFrame."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Ensure 'data' and 'label' keys are present in each item
        labeled_data = [(item["data"], item["label"]) for item in data if "data" in item and "label" in item]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame(columns=["data", "labels"])


def load_list_from_json(filename):
    """Load a list of strings from a JSON file."""
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            return set(json.load(file))
    return set()


def save_list_to_json(filename, data_list):
    """Save a list of strings to a JSON file."""
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(list(data_list), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions for a list of words."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [list(spell.candidates(word)) for word in words_to_check]
    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer for character n-grams."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    spell = SpellChecker(language="en")
    
    # Load custom dictionary and misclassified samples as sets
    custom_dictionary = load_list_from_json(custom_dictionary_filename)
    misclassified_samples = load_list_from_json(misclassified_samples_filename)

    misclassification_threshold = 5
    epochs = 0

    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df.empty or 'labels' not in labeled_df.columns or 'data' not in labeled_df.columns:
            print("Error: Labeled data is empty or missing required columns.")
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)
        model = train_model(X_train_vectorized, all_y_train)

        words_to_check = ["kng", "hgost", "mney", "hllo", "temle", "lon", "aler"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        new_misclassified = [
            word for word, prediction in zip(words_to_check, predictions) if prediction == 0
        ]
        misclassified_samples.update(new_misclassified)

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    custom_dictionary.update(new_vocabulary)
    save_list_to_json(custom_dictionary_filename, custom_dictionary)

    # Save misclassified samples
    save_list_to_json(misclassified_samples_filename, misclassified_samples)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(words_to_check, corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()


Original: kng, Corrected: king, Suggestions: ['king', 'keg']
Original: hgost, Corrected: ghost, Suggestions: ['host', 'ghost']
Original: mney, Corrected: money, Suggestions: ['money']
Original: hllo, Corrected: hello, Suggestions: ['hello', 'halo', 'hallo', 'hollo']
Original: temle, Corrected: temple, Suggestions: ['temple']
Original: lon, Corrected: on, Suggestions: ['ton', 'won', 'lox', 'long', 'lion', 'on', 'non', 'loan', 'son', 'hon', 'lone', 'log', 'yon', 'ln', 'lop', 'loon', 'ion', 'eon', 'lob', 'loin', 'loo', 'lorn', 'low', 'lo', 'don', 'con', 'lot']
Original: aler, Corrected: alert, Suggestions: ['taler', 'alar', 'alert', 'ale', 'alder', 'alee', 'ales', 'aver', 'alter', 'baler', 'haler', 'paler', 'abler']
Training loop finished.


In [6]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file and return it as a DataFrame."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Ensure 'data' and 'label' keys are present in each item
        labeled_data = [(item["data"], item["label"]) for item in data if "data" in item and "label" in item]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame(columns=["data", "labels"])


def load_list_from_json(filename):
    """Load a list of strings from a JSON file."""
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            return set(json.load(file))
    return set()


def save_list_to_json(filename, data_list):
    """Save a list of strings to a JSON file."""
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(list(data_list), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions for a list of words."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [list(spell.candidates(word)) for word in words_to_check]
    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer for character n-grams."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    spell = SpellChecker(language="fr")
    
    # Load custom dictionary and misclassified samples as sets
    custom_dictionary = load_list_from_json(custom_dictionary_filename)
    misclassified_samples = load_list_from_json(misclassified_samples_filename)

    misclassification_threshold = 5
    epochs = 0

    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df.empty or 'labels' not in labeled_df.columns or 'data' not in labeled_df.columns:
            print("Error: Labeled data is empty or missing required columns.")
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)
        model = train_model(X_train_vectorized, all_y_train)

        words_to_check = ["Bore","Lre"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        new_misclassified = [
            word for word, prediction in zip(words_to_check, predictions) if prediction == 0
        ]
        misclassified_samples.update(new_misclassified)

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    custom_dictionary.update(new_vocabulary)
    save_list_to_json(custom_dictionary_filename, custom_dictionary)

    # Save misclassified samples
    save_list_to_json(misclassified_samples_filename, misclassified_samples)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(words_to_check, corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()


Original: Bore, Corrected: boire, Suggestions: ['bure', 'borne', 'borde', 'boxe', 'pore', 'dore', 'fore', 'bord', 'boire', 'boue']
Original: Lre, Corrected: le, Suggestions: ['lie', 'lire', 'lue', 'ère', 'are', 'lyre', 'le']
Training loop finished.


In [7]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score
from spellchecker import SpellChecker
import os
import joblib

def load_labeled_data(filename):
    """Load labeled data from a JSON file and return it as a DataFrame."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Ensure 'data' and 'label' keys are present in each item
        labeled_data = [(item["data"], item["label"]) for item in data if "data" in item and "label" in item]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame(columns=["data", "labels"])


def load_list_from_json(filename):
    """Load a list of strings from a JSON file."""
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            return set(json.load(file))
    return set()


def save_list_to_json(filename, data_list):
    """Save a list of strings to a JSON file."""
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(list(data_list), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions for a list of words."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [list(spell.candidates(word)) for word in words_to_check]
    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer for character n-grams."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"
    model_filename = "naive_bayes_model.pkl"

    spell = SpellChecker(language="fr")
    
    # Load custom dictionary and misclassified samples as sets
    custom_dictionary = load_list_from_json(custom_dictionary_filename)
    spell.word_frequency.load_words(custom_dictionary)

    misclassified_samples = load_list_from_json(misclassified_samples_filename)

    misclassification_threshold = 5
    epochs = 0

    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df.empty or 'labels' not in labeled_df.columns or 'data' not in labeled_df.columns:
            print("Error: Labeled data is empty or missing required columns.")
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)
        model = train_model(X_train_vectorized, all_y_train)

        # Cross-validation for better accuracy estimation
        cv_scores = cross_val_score(model, X_train_vectorized, all_y_train, cv=5)
        print(f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}")

        words_to_check = ["Bore", "Lre"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        new_misclassified = [
            word for word, prediction in zip(words_to_check, predictions) if prediction == 0
        ]
        misclassified_samples.update(new_misclassified)

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the trained model
    joblib.dump(model, model_filename)

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    custom_dictionary.update(new_vocabulary)
    save_list_to_json(custom_dictionary_filename, custom_dictionary)

    # Save misclassified samples
    save_list_to_json(misclassified_samples_filename, misclassified_samples)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(words_to_check, corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()


ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=3.

In [9]:
import json
import os

import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from spellchecker import SpellChecker

from sklearn.model_selection import cross_val_score, StratifiedKFold


def load_labeled_data(filename):
    """Load labeled data from a JSON file and return it as a DataFrame."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Ensure 'data' and 'label' keys are present in each item
        labeled_data = [
            (item["data"], item["label"])
            for item in data
            if "data" in item and "label" in item
        ]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])

    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame(columns=["data", "labels"])


def load_list_from_json(filename):
    """Load a list of strings from a JSON file."""
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            return set(json.load(file))
    return set()


def save_list_to_json(filename, data_list):
    """Save a list of strings to a JSON file."""
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(list(data_list), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions for a list of words."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [list(spell.candidates(word)) for word in words_to_check]
    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer for character n-grams."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"
    model_filename = "naive_bayes_model.pkl"

    spell = SpellChecker(language="fr")

    # Load custom dictionary and misclassified samples as sets
    custom_dictionary = load_list_from_json(custom_dictionary_filename)
    spell.word_frequency.load_words(custom_dictionary)

    misclassified_samples = load_list_from_json(misclassified_samples_filename)

    misclassification_threshold = 5
    epochs = 0

    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if (
            labeled_df.empty
            or "labels" not in labeled_df.columns
            or "data" not in labeled_df.columns
        ):
            print("Error: Labeled data is empty or missing required columns.")
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)
        model = train_model(X_train_vectorized, all_y_train)

        # Find the minimum number of samples in any class
        min_class_size = min([sum(all_y_train == class_label) for class_label in set(all_y_train)])

        # Set n_splits to the minimum of 3 or the smallest class size
        n_splits = min(3, min_class_size)

        if n_splits > 1:
            cv = StratifiedKFold(n_splits=n_splits)
            cv_scores = cross_val_score(model, X_train_vectorized, all_y_train, cv=cv)
            print(f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}")
        else:
            print("Not enough data to perform cross-validation.")

        # # Adjust the number of splits for cross-validation
        # n_splits = min(5, len(all_X_train))
        # if n_splits > 1:
        #     cv_scores = cross_val_score(
        #         model, X_train_vectorized, all_y_train, cv=n_splits
        #     )
        #     print(
        #         f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}"
        #     )
        # else:
        #     print("Not enough samples for cross-validation.")

        words_to_check = ["Bore", "Lre"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        new_misclassified = [
            word
            for word, prediction in zip(words_to_check, predictions)
            if prediction == 0
        ]
        misclassified_samples.update(new_misclassified)

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the trained model
    joblib.dump(model, model_filename)

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    custom_dictionary.update(new_vocabulary)
    save_list_to_json(custom_dictionary_filename, custom_dictionary)

    # Save misclassified samples
    save_list_to_json(misclassified_samples_filename, misclassified_samples)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(
        words_to_check, corrected_words, suggestions
    ):
        print(
            f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}"
        )

    print("Training loop finished.")


if __name__ == "__main__":
    main()

Not enough data to perform cross-validation.
Cross-validation scores: [1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cross-validation scores: [1. 1. 1.], Mean score: 1.0
Cros

In [ ]:
import json
import os

import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from spellchecker import SpellChecker


def load_labeled_data(filename):
    """Load labeled data from a JSON file and return it as a DataFrame."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Ensure 'data' and 'label' keys are present in each item
        labeled_data = [
            (item["data"], item["label"])
            for item in data
            if "data" in item and "label" in item
        ]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])

    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame(columns=["data", "labels"])


def load_list_from_json(filename):
    """Load a list of strings from a JSON file."""
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            return set(json.load(file))
    return set()


def save_list_to_json(filename, data_list):
    """Save a list of strings to a JSON file."""
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(list(data_list), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions for a list of words."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [list(spell.candidates(word)) for word in words_to_check]
    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer for character n-grams."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"
    model_filename = "naive_bayes_model.pkl"

    spell = SpellChecker(language="fr")

    # Load custom dictionary and misclassified samples as sets
    custom_dictionary = load_list_from_json(custom_dictionary_filename)
    spell.word_frequency.load_words(custom_dictionary)

    misclassified_samples = load_list_from_json(misclassified_samples_filename)

    misclassification_threshold = 5
    epochs = 0

    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if (
            labeled_df.empty
            or "labels" not in labeled_df.columns
            or "data" not in labeled_df.columns
        ):
            print("Error: Labeled data is empty or missing required columns.")
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)
        model = train_model(X_train_vectorized, all_y_train)

        # Find the minimum number of samples in any class
        min_class_size = min(
            [sum(all_y_train == class_label) for class_label in set(all_y_train)]
        )

        # Set n_splits to the minimum of 3 or the smallest class size
        n_splits = min(3, min_class_size)

        if n_splits > 1:
            cv = StratifiedKFold(n_splits=n_splits)
            cv_scores = cross_val_score(model, X_train_vectorized, all_y_train, cv=cv)
            print(
                f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}"
            )
        else:
            print("Not enough data to perform cross-validation.")

        words_to_check = ["Bore", "Lre"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        new_misclassified = [
            word
            for word, prediction in zip(words_to_check, predictions)
            if prediction == 0
        ]
        misclassified_samples.update(new_misclassified)

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the trained model
    joblib.dump(model, model_filename)

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    custom_dictionary.update(new_vocabulary)
    save_list_to_json(custom_dictionary_filename, custom_dictionary)

    # Save misclassified samples
    save_list_to_json(misclassified_samples_filename, misclassified_samples)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(
        words_to_check, corrected_words, suggestions
    ):
        print(
            f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}"
        )

    print("Training loop finished.")


if __name__ == "__main__":
    main()